# Biodiversity Studies in Lancanshire, UK (part 2)

We have created Nested Meshes which are now stored in the Postgis backend. 
Also, We have selected a bigger area and we had retrieve the data from the GBIF API. 
*Remember that we have changed the MESHTABLES variable in the settings.py file to work in another region*.

We will proceed now to inspect the Gridded Taxonomy, using the stored mesh.
After that We'll insert it into the Graph Database.


The polygon close to Carnforth gave us some points but all of them with the same geographical coordinates. 
For this reason we decided to extend the AOI (Area of Interest) in order to get more spatial variability (and data).

Our polygon looked like this.

![Carnforth](carnford_poly_points.png "The region")


## Expanding the region.
We increased it to cover a much bigger area.
Here we can see the new points distrtibuted on both sides of the Kent River delta.
The back squares shows the cells in which we will perform the analysis. The Carnforth grove is surrounded by the yellow polygon.

![Startig region](area1.png "The region")


## Load the data.

In [38]:
#First thing first. Load all the biospytial suit
import sys
import django
%matplotlib inline
sys.path.append('/apps')
django.setup()
from gbif.taxonomy import GriddedTaxonomy
from gbif.models import Occurrence

#### Remember how we build the polygon

In [39]:
#from mesh import tools as tl
#windemere = (-2.905,54.376)
#carnforth = (-2.835,54.179) 
#dd = tl.create_rectangle_from_two_points((0,0),(1,1),with_center=windemere)
#polygon = dd['polygon']
square1 = "POLYGON((-2.89749999999999996 54.11650000000000205,-2.89749999999999996 54.24150000000000205,-2.77249999999999996 54.24150000000000205,-2.77249999999999996 54.11650000000000205,-2.89749999999999996 54.11650000000000205))"

In [40]:
## We need to convert the type. It'll be fixed in the future
from django.contrib.gis.geos import GEOSGeometry
polygon = GEOSGeometry(square1)

Now we'll summon the occurrences found here (yes, the newly added).

In [41]:
biosphere = Occurrence.objects.filter(geom__within=polygon)

### How many data here ?

In [42]:
biosphere.count()


5816

### Load the data in a Grid

In [43]:
from mesh.models import initMesh
m10 = initMesh(10)


DEBUG Changed MESH TABLE SPACE


In [44]:
m10_sub = m10.objects.filter(cell__intersects=polygon)

In [45]:
m10_sub.count()

4225

In [34]:
ggg = GriddedTaxonomy?

In [46]:
%time ggg = GriddedTaxonomy(biosphere,m10_sub,"carnforth512")

CPU times: user 13.9 s, sys: 124 ms, total: 14.1 s
Wall time: 14 s


# Environmental variables

Some remarks on taking environmental variables

In [49]:
from django.contrib.gis.db.models.fields import RasterField
from raster_api.models import intersectWith, raster_models
RasterField.register_lookup(intersectWith)

raster_api.models.intersectWith

In [50]:
raster_models

[raster_api.models.Precipitation,
 raster_api.models.SolarRadiation,
 raster_api.models.MeanTemperature,
 raster_api.models.MinTemperature,
 raster_api.models.MaxTemperature,
 raster_api.models.VaporPressure,
 raster_api.models.WindSpeed]

In [51]:
mtemp = raster_models[2]

In [63]:
mtemp.objects.filter(rast__intersect_with=polygon)

<QuerySet []>

In [54]:
from raster_api.tools import RasterData

In [60]:
imgtemp = RasterData(mtemp,polygon)

In [61]:
imgtemp.getRaster()

ERROR Could not extract Raster data from aggregation
ERROR:biospytial.raster_api.tools:Could not extract Raster data from aggregation
Exception django.contrib.gis.gdal.error.GDALException: GDALException('Invalid pointer returned from "GDALClose"',) in 

In [59]:
imgtemp.display_field()

ERROR No raster data associated with specified band. Run getRaster or processDEM first
ERROR:biospytial.raster_api.tools:No raster data associated with specified band. Run getRaster or processDEM first


AttributeError: 'NoneType' object has no attribute 'title'

## I need to load bioclim globally (angry)

In [66]:
ggg.createShapefile(option='shannon',store='carnford1')

INFO [biospytial.gbif.taxonomy.GriddedTaxonomy]
 Creating Shapefile shannon@carnford1
INFO:biospytial.gbif.taxonomy:[biospytial.gbif.taxonomy.GriddedTaxonomy]
 Creating Shapefile shannon@carnford1
INFO [biospytial.gbif.taxonomy.GriddedTaxonomy]
 Shapefile Created in carnford1/shannon
INFO:biospytial.gbif.taxonomy:[biospytial.gbif.taxonomy.GriddedTaxonomy]
 Shapefile Created in carnford1/shannon


True

These are the results.

![Startig region](shannon.png "The region")


### What! Only 2 occurrences ?!

In [68]:
carnforth_occurrences[:10]

[<Occurrence: <GBIF Occurrence: 1202678246  scientific_name: Periparus ater (Linnaeus, 1758)>
  Kingdom: Animalia 
 ,	 Phylum: Chordata 
 ,	 	 Order: Passeriformes,
  	 	 	 Class: Aves, 
  	 	 	 	 Family: Paridae, 
  	 	 	 	 	 Location: s<\GBIF Occurrence>>,
 <Occurrence: <GBIF Occurrence: 1202679548  scientific_name: Turdus merula Linnaeus, 1758>
  Kingdom: Animalia 
 ,	 Phylum: Chordata 
 ,	 	 Order: Passeriformes,
  	 	 	 Class: Aves, 
  	 	 	 	 Family: Turdidae, 
  	 	 	 	 	 Location: s<\GBIF Occurrence>>,
 <Occurrence: <GBIF Occurrence: 1202679575  scientific_name: Phylloscopus trochilus (Linnaeus, 1758)>
  Kingdom: Animalia 
 ,	 Phylum: Chordata 
 ,	 	 Order: Passeriformes,
  	 	 	 Class: Aves, 
  	 	 	 	 Family: Phylloscopidae, 
  	 	 	 	 	 Location: s<\GBIF Occurrence>>,
 <Occurrence: <GBIF Occurrence: 1202679571  scientific_name: Larus marinus Linnaeus, 1758>
  Kingdom: Animalia 
 ,	 Phylum: Chordata 
 ,	 	 Order: Charadriiformes,
  	 	 	 Class: Aves, 
  	 	 	 	 Family: Larida

In [69]:
o = carnforth_occurrences[0]

In [72]:
o.geom.wkt

u'POINT (-2.82074 54.17848)'

In [73]:
points = map(lambda o : o.geom.wkt, carnforth_occurrences)

In [87]:
points[:10]

[u'POINT (-2.82074 54.17848)',
 u'POINT (-2.82074 54.17848)',
 u'POINT (-2.82074 54.17848)',
 u'POINT (-2.82074 54.17848)',
 u'POINT (-2.82074 54.17848)',
 u'POINT (-2.82074 54.17848)',
 u'POINT (-2.82074 54.17848)',
 u'POINT (-2.82074 54.17848)',
 u'POINT (-2.82074 54.17848)',
 u'POINT (-2.82074 54.17848)']

No, just that everybody registered that same point! 
Don't do that pleasEE!

Let's see if we can get more data from there

In [77]:
more_carnforth = pop.getAllOccurrences(carnforth,offset=32)